# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key se ve muy bien")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
openai = OpenAI()

API key se ve muy bien


In [4]:
#Prompt del sistema
system_prompt = "Eres un experto en LLM, inteligencia artificial y python \
el cual se le van a proporcionar una o varias preguntas técnicas relacionadas a los temas a los que eres experto \
y tienes que responder de tal manera que un estudiante que es muy novato y apenas está aprendiendo entienda"

In [5]:
# here is the question; type over this to ask something new

question = """
Por favor explica que hace este código y porque:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [6]:
# Prompt del usuario
user_prompt = f"Responde la siguiente pregunta en formato markdown: {question}"

In [7]:
# Crear la repuesta sin streaming
def crear_respuesta():
    response = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = [
            {"role":"system", "content": system_prompt},
            {"role":"user", "content": user_prompt}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [8]:
crear_respuesta()

¡Claro! Vamos a desglosar este código paso a paso para que puedas entenderlo.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Explicación del código

1. **Contexto**: Supongamos que `books` es una lista de diccionarios. Cada diccionario representa un libro y contiene información sobre el mismo, como el autor, el título, etc. Por ejemplo:

   ```python
   books = [
       {"title": "Libro A", "author": "Autor A"},
       {"title": "Libro B", "author": "Autor B"},
       {"title": "Libro C", "author": None},
   ]
   ```

2. **Comprensión de conjunto**: El fragmento de código `{book.get("author") for book in books if book.get("author")}` es una **comprensión de conjunto**. Esto significa que estamos creando un conjunto (que es una colección de elementos únicos) a partir de la lista `books`.

   - `for book in books`: Esto itera sobre cada libro en la lista `books`.
   - `book.get("author")`: Este método intenta obtener el valor de la clave "author" de cada libro. Si la clave no existe o su valor es `None`, no se considera.
   - `if book.get("author")`: Aquí filtramos para asegurarnos de que solo estamos recogiendo autores que realmente existen (es decir, que no son `None` o una cadena vacía).

   Entonces, si el conjunto `authors` resultante de la anterior comprensión de conjunto es `{"Autor A", "Autor B"}`, significa que hemos recogido solo los autores válidos.

3. **`yield from`**: Esta parte `yield from` se utiliza dentro de una función generadora. Permite que una función generadora produzca valores de otro generador o iterable. En este caso, está generando todos los valores del conjunto de autores que hemos creado.

   Esto es útil porque permite que la función que contiene este código pueda ir devolviendo uno a uno los autores sin necesidad de cargar todo en memoria a la vez, lo que es más eficiente.

### Resumen

- **Qué hace**: El código recoge todos los autores de los libros en la lista `books`, ignorando aquellos donde el autor no está disponible, y devuelve cada uno de ellos, uno a uno, a través de una función generadora.
  
- **Por qué**: Usamos un conjunto para asegurarnos de que no haya duplicados en los autores. Y `yield from` permite que la función generadora produzca estos autores de forma eficiente.

Con esto, espero que tengas una mejor comprensión de lo que hace este código y por qué se utiliza de esta manera. ¡Si tienes más preguntas, no dudes en preguntar!

In [9]:
# Get gpt-4o-mini to answer, with streaming
def stream_respuesta():
    stream = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = [
            {"role":"system", "content": system_prompt},
            {"role":"user", "content": user_prompt}
        ],
        stream = True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        #response = response.replace("```", "").replace("markdown","")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [10]:
stream_respuesta()

Claro, vamos a desglosar este código línea por línea para que sea más fácil de entender.

### Explicación del Código

```python
yield from {book.get("author") for book in books if book.get("author")}
```

1. **`{...}`**: Al inicio vemos llaves `{}`. Esto indica que estamos creando un conjunto (set en inglés). Un conjunto es una colección que no admite duplicados, es decir, solo guardará valores únicos.

2. **`for book in books`**: Esta parte es un bucle que itera (o pasa por cada elemento) de la lista `books`. Aquí, `books` debería ser una lista (o cualquier tipo de iterable) que contiene elementos, y cada uno de esos elementos se asigna a la variable `book` en cada iteración.

3. **`book.get("author")`**: Esto intenta obtener el valor asociado a la clave `"author"` de cada `book`. `get` es un método que devuelve el valor de la clave especificada si existe, de lo contrario devuelve `None`. Este método es útil porque evita errores si la clave no existe.

4. **`if book.get("author")`**: Esta es una condición que se asegura de que solo se añadan al conjunto aquellos libros que efectivamente tienen un autor definido. Si `book.get("author")` devuelve `None` (lo que significa que no hay autor), entonces ese libro no se incluye en el conjunto.

5. **`yield from ...`**: La palabra clave `yield` se utiliza en las funciones generadoras en Python. Al usar `yield from`, estamos delegando la producción de los valores generados por el conjunto que hemos creado. Esto significa que, en vez de devolver el conjunto completo de una vez, se van devolviendo elementos uno por uno.

### Resumen

En resumen, este código crea un conjunto de autores únicos a partir de una lista de libros. Solo se incluyen a aquellos libros que realmente tienen un autor especificado. Luego, con `yield from`, este conjunto se produce de manera que se puede iterar a través de él de forma eficiente.

### ¿Por qué es útil?

- **Eficiencia**: Al usar un conjunto, nos aseguramos de no tener duplicados. Por ejemplo, si varios libros son del mismo autor, solo se devolverá ese autor una vez.
- **Manejo de valores faltantes**: Ignora libros que no tienen autor, evitando errores o valores indeseados en la colección resultante.

Espero que esto te ayude a entender mejor lo que hace este código. Si tienes más preguntas, ¡no dudes en preguntar!

In [15]:
# Get Llama 3.2 to answer
import ollama

def stream_respuesta_ollama():
    stream = ollama.chat(
        model = MODEL_LLAMA,
        messages = [
            {"role":"system", "content": system_prompt},
            {"role":"user", "content": user_prompt}
        ],
        stream = True
    )
    response = ''
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk['message']['content'] or ''
        #response = response.replace("```", "").replace("markdown","")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [16]:
stream_respuesta_ollama()

**Análisis del Código**
=====================================

El código proporcionado es un ejemplo de uso de la función `yield from` en Python, que se utiliza para generar una secuencia de valores.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Componentes del Código

*   **`{book.get("author") for book in books if book.get("author")}`**: Esta es una expresión generadora que utiliza un método de listado (list comprehension) para crear una secuencia de valores.
    *   `for book in books`: Itera sobre cada elemento en la lista `books`.
    *   `if book.get("author")`: Filtera los elementos para solo incluir aquellos que tengan un valor asociado a la clave `"author"`. Si el valor es `None`, se considerará como falso y no será incluido en la secuencia.
    *   `book.get("author")`: Extrae el valor asociado a la clave `"author"` del objeto `book`.

### Uso de `yield from`

*   **`yield from {...}`**: Esta es una forma de implementar un generador que permite que otra función devuelva una secuencia de valores sin necesidad de crear una lista en memoria.
    *   El `yield from` permite que la secuencia creada por el método de listado se convierta en un generador, lo que significa que se puede utilizar para producir un número infinito de valores en lugar de crear una lista completa en memoria.

### Efectos del Código

En resumen, este código:

1.  Itera sobre cada elemento en la lista `books`.
2.  Filtra los elementos para solo incluir aquellos que tengan un valor asociado a la clave `"author"`.
3.  Extrae el valor asociado a la clave `"author"` de cada elemento filtrado.
4.  Genera una secuencia de valores utilizando el método de listado.

### Por Qué se Utiliza

Se utiliza este código cuando se necesita generar una secuencia de valores que contienen información específica (en este caso, los autores de libros) sin necesidad de crear una lista completa en memoria. Esto es especialmente útil cuando se trabaja con grandes conjuntos de datos y se necesita optimizar el uso de la memoria.

**Ejemplo Aplicado**
--------------------

Supongamos que tienes una lista de libros con información sobre sus autores, y quieres generar una secuencia de los nombres de los autores. Puedes utilizar este código:

```python
books = [
    {"title": "Libro 1", "author": "Autor A"},
    {"title": "Libro 2", "author": None},
    {"title": "Libro 3", "author": "Autor B"}
]

autores = yield from {book.get("author") for book in books if book.get("author")}
print(autores)  # Salida: ['Autor A', 'Autor B']
```

En este ejemplo, el código se utiliza para generar una secuencia de los nombres de los autores de los libros que tienen un valor asociado a la clave `"author"`.